In [ ]:
from PIL import Image
import numpy as np
from patchify import patchify
import os
Image.MAX_IMAGE_PIXELS = None # disable protection from potential decompression bomb DOS attack
image = Image.open("TitheTrainingMaskVisable.tif")
#display(image)
image_array = np.array(image)
print(image_array.shape)

(7878, 10646)


In [4]:
patch_size = 256
# Calculate padding needed
h, w = image_array.shape
c = 1

pad_h = (patch_size - h % patch_size) % patch_size
pad_w = (patch_size - w % patch_size) % patch_size
print(f"Padding needed: height={pad_h}, width={pad_w}")

# Pad the mask directly
padded_mask = np.pad(image_array, 
                     ((0, pad_h), (0, pad_w)), 
                     mode='constant', 
                     constant_values=0)


Padding needed: height=58, width=106


In [ ]:
# display mask for debugging:
# Image.fromarray(padded_mask.astype(np.uint8)).show()

In [7]:
patches=patchify(padded_mask,(patch_size,patch_size),step=patch_size)
print(patches.shape)

(31, 42, 256, 256)


In [9]:
output_dir = f"mask_patches_{patch_size}"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print(f"Mask patches shape: {mask_patches.shape}")
print(f"Will create {mask_patches.shape[0] * mask_patches.shape[1]} patches")

for i in range(mask_patches.shape[0]):
    for j in range(mask_patches.shape[1]):
        # Handle different possible shapes from patchify
        patch = mask_patches[i, j]
        if len(patch.shape) > 2:
            patch = np.squeeze(patch)
        
        # Debug first few patches
        if i == 0 and j < 3:
            print(f"Patch {i},{j}: shape={patch.shape}, dtype={patch.dtype}")
            print(f"  Values: min={patch.min()}, max={patch.max()}")
            print(f"  Unique values: {np.unique(patch)}")
        
        # Ensure proper data type for saving
        patch_uint8 = patch.astype(np.uint8)
        patch_img = Image.fromarray(patch_uint8)
        patch_filename = f"patch_{i}_{j}.png"
        patch_img.save(os.path.join(output_dir, patch_filename))

print(f"Saved all patches to {output_dir}")


Mask patches shape: (31, 42, 256, 256)
Will create 1302 patches
Patch 0,0: shape=(256, 256), dtype=uint8
  Values: min=0, max=0
  Unique values: [0]
Patch 0,1: shape=(256, 256), dtype=uint8
  Values: min=0, max=0
  Unique values: [0]
Patch 0,2: shape=(256, 256), dtype=uint8
  Values: min=0, max=0
  Unique values: [0]
Saved all patches to mask_patches_256


this is cleaner code, but it does not work for some reason:

In [ ]:
output_dir = f"mask_patches_{patch_size}"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for i in range(patches.shape[0]):
    for j in range(patches.shape[1]):
        patch = patches[i, j, 0]
        patch_img = Image.fromarray(patch.astype(np.uint8))
        patch_filename = f"patch_{i}_{j}.png"
        patch_img.save(os.path.join(output_dir, patch_filename))